In [1]:
!wget https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/test.csv.zip
!wget https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/train.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip

--2018-02-17 16:34:29--  https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/test.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24577258 (23M) [application/zip]
Saving to: ‘test.csv.zip’

test.csv.zip        100%[===================>]  23.44M  96.3MB/s    in 0.2s    

2018-02-17 16:34:35 (96.3 MB/s) - ‘test.csv.zip’ saved [24577258/24577258]

--2018-02-17 16:34:36--  https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/train.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27619914 (26M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv.z

In [7]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('./train.csv').fillna(' ')
test = pd.read_csv('./test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])


In [8]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=20000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [9]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=20000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [10]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [11]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

CV score for class toxic is 0.9786736029840394
CV score for class severe_toxic is 0.9888048107362089
CV score for class obscene is 0.9903493503589947
CV score for class threat is 0.9890177602746655
CV score for class insult is 0.982841518026043
CV score for class identity_hate is 0.9830985377349876


In [ ]:
len(train_features.toarray()[0])